In [ ]:
# [1. 음성 파일 -> text 추출]
!pip install git+https://github.com/openai/whisper.git

^C


  Cloning https://github.com/openai/whisper.git to c:\users\user\appdata\local\temp\pip-req-build-w_gwenvj
  Resolved https://github.com/openai/whisper.git to commit 173ff7dd1d9fb1c4fddea0d41d704cfefeb8908c
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached torch-2.5.1-cp39-cp39-win_amd64.whl.metadata (28 kB)
  Using cached tqdm-4.67.0-py3-none-any.whl.metadata (57 kB)
  Using cached more_itertools-10.5.0-py3-none-any.whl.metadata (36 kB)
  Using cached tiktoken-0.8.0-cp39-cp39-win_amd64.whl.metadata (6.8 kB)
  Using cached regex-2024.11.6-cp39-cp39-win_amd64.whl.metadata (41 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached networkx-3.2.1-py3-none-any.whl.metada

  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git 'C:\Users\user\AppData\Local\Temp\pip-req-build-w_gwenvj'


In [2]:
import os
import glob
import whisper

# Whisper 모델 로드
model = whisper.load_model("small")

# 부모 폴더 경로 설정
parent_folder = r"./test_audio/child_original"  # 음성 파일이 있는 폴더 경로

# 지원되는 파일 형식
audio_extensions = ('.wav')

# 폴더 내 모든 음성 파일 불러오기
audio_files = [f for f in glob.glob(os.path.join(parent_folder, "*")) if f.endswith(audio_extensions)]

# 음성 파일 하나씩 처리
for audio_file in audio_files:
    print(f"처리 중: {audio_file}")
    try:
        # 텍스트 추출
        result = model.transcribe(audio_file)
        extracted_text = result["text"]

        # 결과 출력
        print(f"추출된 텍스트 ({os.path.basename(audio_file)}):")
        print(extracted_text)
        print("-" * 50)  # 구분선
    except Exception as e:
        print(f"오류 발생 ({audio_file}): {e}")


c:\Users\user\Anaconda3\envs\epa_project\lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=de

처리 중: ./test_audio/child_original\jaemin_1.wav


c:\Users\user\Anaconda3\envs\epa_project\lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


추출된 텍스트 (jaemin_1.wav):
 My grandfather and my mom are the same.
--------------------------------------------------
처리 중: ./test_audio/child_original\jaemin_2.wav
추출된 텍스트 (jaemin_2.wav):
 It rained a lot when we went to Jeju-do.
--------------------------------------------------
처리 중: ./test_audio/child_original\jaemin_3.wav
추출된 텍스트 (jaemin_3.wav):
 I want to like you more than you do.
--------------------------------------------------
처리 중: ./test_audio/child_original\jaemin_4.wav
추출된 텍스트 (jaemin_4.wav):
 My daughter cried loudly.
--------------------------------------------------
처리 중: ./test_audio/child_original\jaemin_5.wav
추출된 텍스트 (jaemin_5.wav):
 It stopped raining.
--------------------------------------------------
처리 중: ./test_audio/child_original\jaemin_6.wav
추출된 텍스트 (jaemin_6.wav):
 He does not talk much.
--------------------------------------------------


In [ ]:
# [2. 출력된 텍스트로 tortoise-TTS highquality 생성]

In [11]:
import os
import librosa
import numpy as np
import pandas as pd
from google.cloud import speech
import soundfile as sf
from dotenv import load_dotenv

# Google Cloud Speech-to-Text API 설정
load_dotenv()
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv("KEY_PATH")
speech_client = speech.SpeechClient()

# 평가 함수
def evaluate_audio(user_file, reference_file):
    results = []
    try:
        # Load audio files
        user_y, sr = librosa.load(user_file)
        ref_y, _ = librosa.load(reference_file)

        # 1) 음율적 오류 (Pitch, Rhythm)
        user_pitches, _ = librosa.piptrack(y=user_y, sr=sr)
        ref_pitches, _ = librosa.piptrack(y=ref_y, sr=sr)
        user_pitch_pattern = [np.max(user_pitches[:, i]) for i in range(len(user_pitches[0])) if np.max(user_pitches[:, i]) > 0]
        ref_pitch_pattern = [np.max(ref_pitches[:, i]) for i in range(len(ref_pitches[0])) if np.max(ref_pitches[:, i]) > 0]

        min_length = min(len(user_pitch_pattern), len(ref_pitch_pattern))
        pitch_contour_similarity = np.corrcoef(user_pitch_pattern[:min_length], ref_pitch_pattern[:min_length])[0, 1]
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'pitch_contour_similarity',
            'category': '음율적 오류',
            'value': pitch_contour_similarity
        })

        user_times = librosa.frames_to_time(range(len(user_pitches[0])), sr=sr)
        ref_times = librosa.frames_to_time(range(len(ref_pitches[0])), sr=sr)
        user_intervals = np.diff(user_times)
        ref_intervals = np.diff(ref_times)
        rhythm_similarity = np.abs(user_intervals[:min_length] - ref_intervals[:min_length]).mean()
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'rhythm_similarity',
            'category': '음율적 오류',
            'value': rhythm_similarity
        })

        # 2) 정확성 (Pitch 평균, RMS)
        user_avg_pitch = np.mean(user_pitch_pattern)
        ref_avg_pitch = np.mean(ref_pitch_pattern)
        pitch_difference = np.abs(user_avg_pitch - ref_avg_pitch)
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'pitch_difference',
            'category': '정확성',
            'value': pitch_difference
        })

        user_rms = librosa.feature.rms(y=user_y).mean()
        ref_rms = librosa.feature.rms(y=ref_y).mean()
        rms_difference = np.abs(user_rms - ref_rms)
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'rms_difference',
            'category': '정확성',
            'value': rms_difference
        })

        # 3) 유창성 (Speech Rate, Pauses)
        user_duration = librosa.get_duration(y=user_y, sr=sr)
        ref_duration = librosa.get_duration(y=ref_y, sr=sr)
        speech_rate_difference = np.abs((len(user_pitch_pattern) / user_duration) - (len(ref_pitch_pattern) / ref_duration))
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'speech_rate_difference',
            'category': '유창성',
            'value': speech_rate_difference
        })

        pause_threshold = 0.3
        user_pauses = len([i for i in np.diff(user_times) if i > pause_threshold])
        ref_pauses = len([i for i in np.diff(ref_times) if i > pause_threshold])
        pause_difference = np.abs(user_pauses - ref_pauses)
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'pause_difference',
            'category': '유창성',
            'value': pause_difference
        })

        # 4) 완전성 (Word Omission)
        word_omission = 0  # Simplified for brevity
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'word_omission',
            'category': '완전성',
            'value': word_omission
        })

    except Exception as e:
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'error',
            'category': 'Error',
            'value': str(e)
        })
    
    return results

# 메인 함수
def main(reference_folder, user_folder):
    all_results = []

    for ref_file in os.listdir(reference_folder):
        if ref_file.endswith(".wav"):
            ref_path = os.path.join(reference_folder, ref_file)
            
            for user_file in os.listdir(user_folder):
                if user_file.endswith(".wav"):
                    user_path = os.path.join(user_folder, user_file)
                    all_results.extend(evaluate_audio(user_path, ref_path))

    # DataFrame 생성
    df = pd.DataFrame(all_results)
    print(df)

# 실행
reference_folder = "./test_audio/adult_tts"
user_folder = "./test_audio/adult_original"
main(reference_folder, user_folder)


      user_file reference_file         evaluation_metric category       value
0    yura_1.wav      tts_1.wav  pitch_contour_similarity   음율적 오류    0.147717
1    yura_1.wav      tts_1.wav         rhythm_similarity   음율적 오류    0.000000
2    yura_1.wav      tts_1.wav          pitch_difference      정확성  130.154541
3    yura_1.wav      tts_1.wav            rms_difference      정확성    0.018920
4    yura_1.wav      tts_1.wav    speech_rate_difference      유창성    0.083839
..          ...            ...                       ...      ...         ...
247  yura_6.wav      tts_6.wav          pitch_difference      정확성  364.937134
248  yura_6.wav      tts_6.wav            rms_difference      정확성    0.003422
249  yura_6.wav      tts_6.wav    speech_rate_difference      유창성    6.377279
250  yura_6.wav      tts_6.wav          pause_difference      유창성    0.000000
251  yura_6.wav      tts_6.wav             word_omission      완전성    0.000000

[252 rows x 5 columns]


In [12]:
# [4. 성인 표준음성(TTS MARK) - 아동 진짜사람음성 차이]
import os
import librosa
import numpy as np
import pandas as pd
from google.cloud import speech
import soundfile as sf
from dotenv import load_dotenv

# Google Cloud Speech-to-Text API 설정
load_dotenv()
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv("KEY_PATH")
speech_client = speech.SpeechClient()

# 평가 함수
def evaluate_audio(user_file, reference_file):
    results = []
    try:
        # Load audio files
        user_y, sr = librosa.load(user_file)
        ref_y, _ = librosa.load(reference_file)

        # 1) 음율적 오류 (Pitch, Rhythm)
        user_pitches, _ = librosa.piptrack(y=user_y, sr=sr)
        ref_pitches, _ = librosa.piptrack(y=ref_y, sr=sr)
        user_pitch_pattern = [np.max(user_pitches[:, i]) for i in range(len(user_pitches[0])) if np.max(user_pitches[:, i]) > 0]
        ref_pitch_pattern = [np.max(ref_pitches[:, i]) for i in range(len(ref_pitches[0])) if np.max(ref_pitches[:, i]) > 0]

        min_length = min(len(user_pitch_pattern), len(ref_pitch_pattern))
        pitch_contour_similarity = np.corrcoef(user_pitch_pattern[:min_length], ref_pitch_pattern[:min_length])[0, 1]
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'pitch_contour_similarity',
            'category': '음율적 오류',
            'value': pitch_contour_similarity
        })

        user_times = librosa.frames_to_time(range(len(user_pitches[0])), sr=sr)
        ref_times = librosa.frames_to_time(range(len(ref_pitches[0])), sr=sr)
        user_intervals = np.diff(user_times)
        ref_intervals = np.diff(ref_times)
        rhythm_similarity = np.abs(user_intervals[:min_length] - ref_intervals[:min_length]).mean()
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'rhythm_similarity',
            'category': '음율적 오류',
            'value': rhythm_similarity
        })

        # 2) 정확성 (Pitch 평균, RMS)
        user_avg_pitch = np.mean(user_pitch_pattern)
        ref_avg_pitch = np.mean(ref_pitch_pattern)
        pitch_difference = np.abs(user_avg_pitch - ref_avg_pitch)
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'pitch_difference',
            'category': '정확성',
            'value': pitch_difference
        })

        user_rms = librosa.feature.rms(y=user_y).mean()
        ref_rms = librosa.feature.rms(y=ref_y).mean()
        rms_difference = np.abs(user_rms - ref_rms)
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'rms_difference',
            'category': '정확성',
            'value': rms_difference
        })

        # 3) 유창성 (Speech Rate, Pauses)
        user_duration = librosa.get_duration(y=user_y, sr=sr)
        ref_duration = librosa.get_duration(y=ref_y, sr=sr)
        speech_rate_difference = np.abs((len(user_pitch_pattern) / user_duration) - (len(ref_pitch_pattern) / ref_duration))
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'speech_rate_difference',
            'category': '유창성',
            'value': speech_rate_difference
        })

        pause_threshold = 0.3
        user_pauses = len([i for i in np.diff(user_times) if i > pause_threshold])
        ref_pauses = len([i for i in np.diff(ref_times) if i > pause_threshold])
        pause_difference = np.abs(user_pauses - ref_pauses)
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'pause_difference',
            'category': '유창성',
            'value': pause_difference
        })

        # 4) 완전성 (Word Omission)
        word_omission = 0  # Simplified for brevity
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'word_omission',
            'category': '완전성',
            'value': word_omission
        })

    except Exception as e:
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'error',
            'category': 'Error',
            'value': str(e)
        })
    
    return results

# 메인 함수
def main(reference_folder, user_folder):
    all_results = []

    for ref_file in os.listdir(reference_folder):
        if ref_file.endswith(".wav"):
            ref_path = os.path.join(reference_folder, ref_file)
            
            for user_file in os.listdir(user_folder):
                if user_file.endswith(".wav"):
                    user_path = os.path.join(user_folder, user_file)
                    all_results.extend(evaluate_audio(user_path, ref_path))

    # DataFrame 생성
    df = pd.DataFrame(all_results)
    print(df)

# 실행
reference_folder = "./test_audio/adult_tts"
user_folder = "./test_audio/child_original"
main(reference_folder, user_folder)


        user_file reference_file         evaluation_metric category  \
0    jaemin_1.wav      tts_1.wav  pitch_contour_similarity   음율적 오류   
1    jaemin_1.wav      tts_1.wav         rhythm_similarity   음율적 오류   
2    jaemin_1.wav      tts_1.wav          pitch_difference      정확성   
3    jaemin_1.wav      tts_1.wav            rms_difference      정확성   
4    jaemin_1.wav      tts_1.wav    speech_rate_difference      유창성   
..            ...            ...                       ...      ...   
247  jaemin_6.wav      tts_6.wav          pitch_difference      정확성   
248  jaemin_6.wav      tts_6.wav            rms_difference      정확성   
249  jaemin_6.wav      tts_6.wav    speech_rate_difference      유창성   
250  jaemin_6.wav      tts_6.wav          pause_difference      유창성   
251  jaemin_6.wav      tts_6.wav             word_omission      완전성   

           value  
0       0.080350  
1       0.000000  
2    1255.984619  
3       0.068354  
4       1.827973  
..           ...  
247  1080.0323

In [13]:
import os
import librosa
import numpy as np
import pandas as pd
from google.cloud import speech
import soundfile as sf
from dotenv import load_dotenv

# Google Cloud Speech-to-Text API 설정
load_dotenv()
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv("KEY_PATH")
speech_client = speech.SpeechClient()

# 평가 함수
def evaluate_audio(user_file, reference_file):
    results = []
    try:
        # Load audio files
        user_y, sr = librosa.load(user_file)
        ref_y, _ = librosa.load(reference_file)

        # 1) 음율적 오류 (Pitch, Rhythm)
        user_pitches, _ = librosa.piptrack(y=user_y, sr=sr)
        ref_pitches, _ = librosa.piptrack(y=ref_y, sr=sr)
        user_pitch_pattern = [np.max(user_pitches[:, i]) for i in range(len(user_pitches[0])) if np.max(user_pitches[:, i]) > 0]
        ref_pitch_pattern = [np.max(ref_pitches[:, i]) for i in range(len(ref_pitches[0])) if np.max(ref_pitches[:, i]) > 0]

        min_length = min(len(user_pitch_pattern), len(ref_pitch_pattern))
        pitch_contour_similarity = np.corrcoef(user_pitch_pattern[:min_length], ref_pitch_pattern[:min_length])[0, 1]
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'pitch_contour_similarity',
            'category': '음율적 오류',
            'value': pitch_contour_similarity
        })

        user_times = librosa.frames_to_time(range(len(user_pitches[0])), sr=sr)
        ref_times = librosa.frames_to_time(range(len(ref_pitches[0])), sr=sr)
        user_intervals = np.diff(user_times)
        ref_intervals = np.diff(ref_times)
        rhythm_similarity = np.abs(user_intervals[:min_length] - ref_intervals[:min_length]).mean()
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'rhythm_similarity',
            'category': '음율적 오류',
            'value': rhythm_similarity
        })

        # 2) 정확성 (Pitch 평균, RMS)
        user_avg_pitch = np.mean(user_pitch_pattern)
        ref_avg_pitch = np.mean(ref_pitch_pattern)
        pitch_difference = np.abs(user_avg_pitch - ref_avg_pitch)
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'pitch_difference',
            'category': '정확성',
            'value': pitch_difference
        })

        user_rms = librosa.feature.rms(y=user_y).mean()
        ref_rms = librosa.feature.rms(y=ref_y).mean()
        rms_difference = np.abs(user_rms - ref_rms)
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'rms_difference',
            'category': '정확성',
            'value': rms_difference
        })

        # 3) 유창성 (Speech Rate, Pauses)
        user_duration = librosa.get_duration(y=user_y, sr=sr)
        ref_duration = librosa.get_duration(y=ref_y, sr=sr)
        speech_rate_difference = np.abs((len(user_pitch_pattern) / user_duration) - (len(ref_pitch_pattern) / ref_duration))
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'speech_rate_difference',
            'category': '유창성',
            'value': speech_rate_difference
        })

        pause_threshold = 0.3
        user_pauses = len([i for i in np.diff(user_times) if i > pause_threshold])
        ref_pauses = len([i for i in np.diff(ref_times) if i > pause_threshold])
        pause_difference = np.abs(user_pauses - ref_pauses)
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'pause_difference',
            'category': '유창성',
            'value': pause_difference
        })

        # 4) 완전성 (Word Omission)
        word_omission = 0  # Simplified for brevity
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'word_omission',
            'category': '완전성',
            'value': word_omission
        })

    except Exception as e:
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'error',
            'category': 'Error',
            'value': str(e)
        })
    
    return results

# 메인 함수
def main(reference_folder, user_folder, output_csv):
    all_results = []

    for ref_file in os.listdir(reference_folder):
        if ref_file.endswith(".wav"):
            ref_path = os.path.join(reference_folder, ref_file)
            
            for user_file in os.listdir(user_folder):
                if user_file.endswith(".wav"):
                    user_path = os.path.join(user_folder, user_file)
                    all_results.extend(evaluate_audio(user_path, ref_path))

    # DataFrame 생성
    df = pd.DataFrame(all_results)
    
    # CSV로 저장
    df.to_csv(output_csv, index=False, encoding='utf-8-sig')
    print(f"[INFO] Results saved to {output_csv}")

# 실행
reference_folder = "./test_audio/adult_tts"
user_folder = "./test_audio/adult_original"
output_csv = "./evaluation_results.csv"
main(reference_folder, user_folder, output_csv)

[INFO] Results saved to ./evaluation_results.csv


In [14]:
import os
import librosa
import numpy as np
import pandas as pd
from google.cloud import speech
import soundfile as sf
from dotenv import load_dotenv

# Google Cloud Speech-to-Text API 설정
load_dotenv()
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv("KEY_PATH")
speech_client = speech.SpeechClient()

# 평가 함수
def evaluate_audio(user_file, reference_file):
    results = []
    try:
        # Load audio files
        user_y, sr = librosa.load(user_file)
        ref_y, _ = librosa.load(reference_file)

        # 1) 음율적 오류 (Pitch, Rhythm)
        user_pitches, _ = librosa.piptrack(y=user_y, sr=sr)
        ref_pitches, _ = librosa.piptrack(y=ref_y, sr=sr)
        user_pitch_pattern = [np.max(user_pitches[:, i]) for i in range(len(user_pitches[0])) if np.max(user_pitches[:, i]) > 0]
        ref_pitch_pattern = [np.max(ref_pitches[:, i]) for i in range(len(ref_pitches[0])) if np.max(ref_pitches[:, i]) > 0]

        min_length = min(len(user_pitch_pattern), len(ref_pitch_pattern))
        pitch_contour_similarity = np.corrcoef(user_pitch_pattern[:min_length], ref_pitch_pattern[:min_length])[0, 1]
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'pitch_contour_similarity',
            'category': '음율적 오류',
            'value': pitch_contour_similarity
        })

        user_times = librosa.frames_to_time(range(len(user_pitches[0])), sr=sr)
        ref_times = librosa.frames_to_time(range(len(ref_pitches[0])), sr=sr)
        user_intervals = np.diff(user_times)
        ref_intervals = np.diff(ref_times)
        rhythm_similarity = np.abs(user_intervals[:min_length] - ref_intervals[:min_length]).mean()
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'rhythm_similarity',
            'category': '음율적 오류',
            'value': rhythm_similarity
        })

        # 2) 정확성 (Pitch 평균, RMS)
        user_avg_pitch = np.mean(user_pitch_pattern)
        ref_avg_pitch = np.mean(ref_pitch_pattern)
        pitch_difference = np.abs(user_avg_pitch - ref_avg_pitch)
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'pitch_difference',
            'category': '정확성',
            'value': pitch_difference
        })

        user_rms = librosa.feature.rms(y=user_y).mean()
        ref_rms = librosa.feature.rms(y=ref_y).mean()
        rms_difference = np.abs(user_rms - ref_rms)
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'rms_difference',
            'category': '정확성',
            'value': rms_difference
        })

        # 3) 유창성 (Speech Rate, Pauses)
        user_duration = librosa.get_duration(y=user_y, sr=sr)
        ref_duration = librosa.get_duration(y=ref_y, sr=sr)
        speech_rate_difference = np.abs((len(user_pitch_pattern) / user_duration) - (len(ref_pitch_pattern) / ref_duration))
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'speech_rate_difference',
            'category': '유창성',
            'value': speech_rate_difference
        })

        pause_threshold = 0.3
        user_pauses = len([i for i in np.diff(user_times) if i > pause_threshold])
        ref_pauses = len([i for i in np.diff(ref_times) if i > pause_threshold])
        pause_difference = np.abs(user_pauses - ref_pauses)
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'pause_difference',
            'category': '유창성',
            'value': pause_difference
        })

        # 4) 완전성 (Word Omission)
        word_omission = 0  # Simplified for brevity
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'word_omission',
            'category': '완전성',
            'value': word_omission
        })

    except Exception as e:
        results.append({
            'user_file': os.path.basename(user_file),
            'reference_file': os.path.basename(reference_file),
            'evaluation_metric': 'error',
            'category': 'Error',
            'value': str(e)
        })
    
    return results

# 메인 함수
def main(reference_folder, user_folder, output_csv):
    all_results = []

    for ref_file in os.listdir(reference_folder):
        if ref_file.endswith(".wav"):
            ref_path = os.path.join(reference_folder, ref_file)
            
            for user_file in os.listdir(user_folder):
                if user_file.endswith(".wav"):
                    user_path = os.path.join(user_folder, user_file)
                    all_results.extend(evaluate_audio(user_path, ref_path))

    # DataFrame 생성
    df = pd.DataFrame(all_results)
    
    # CSV로 저장
    df.to_csv(output_csv, index=False, encoding='utf-8-sig')
    print(f"[INFO] Results saved to {output_csv}")

# 실행
reference_folder = "./test_audio/adult_tts"
user_folder = "./test_audio/child_original"
output_csv = "./evaluation_results_child.csv"
main(reference_folder, user_folder, output_csv)


[INFO] Results saved to ./evaluation_results_child.csv
